In [ ]:
梁老师你好：
这里放置了学习过的3个基础机器学习算法的原理部分代码，掌握的比较熟练。这些代码是老师讲义，个人作业的拼接，但是框架是老师给出的。
在学习过程中还有一些数据可视化方面的尝试和代码，以及简单的优化，就不加入这里了。
可以看出这里的算法非常基础，但能够提供给我一些对机器学习的理解。
对其他算法也有了解，但是都有些不完全理解的部分，没有展示，例如svm，不太理解感知基到向量机的转变与对偶性的数学部分，logistic regression的信息熵部分

# KNN

面向对象，KNN具有三个属性: k, X_train, y_train 
有两个方法: fit 和 predict
fit 有两个参数: X_train 和 y_train, 用来更新KNN的属性 X_train, y_train. 返回 KNN对象本身.
predict 有一个参数: X_test, 在预测完 X_test 的标签(类别)后返回所有的预测值

In [13]:
import numpy as np

def eu_dist(u, v):   #欧式距离，计算两个对象的空间距离，最简单的距离衡量
    return np.sqrt(np.sum((u - v) ** 2))


def most_frequent(ar):
    u, indices = np.unique(ar, return_inverse=True)
    return u[np.argmax(np.bincount(indices))]

class KNN:
    def __init__(self, k=3):
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        return self

    def predict(self, X_test):
        predictions = []
        for i, unknown_sample in enumerate(X_test):
            neighbours = []
            for j, observed_sample in enumerate(self.X_train):
                distance = eu_dist(unknown_sample, observed_sample)
                neighbours.append([distance, self.y_train[j]])
            neighbours = np.array(neighbours)
            knn = neighbours[neighbours[:, 0].argsort()][:self.k]
            predictions.append(most_frequent(knn[:, 1]))
        return np.ravel(predictions)

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 


iris = pd.read_csv('bird.csv', delimiter=',')

X = iris[['Sepal length', 'Sepal width',
          'Petal length', 'Petal width']].as_matrix()
y = iris['Species'].as_matrix()

# 数据集分割, 67%的用于训练, 33%的用于测试
# X_train, X_test = 训练样本, 测试样本
# y_train, y_test = 训练目标, 测试目标
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=4200)     #可以设置不同的random state来进行交叉实验  

knn = KNN(k=3)

# 预测测试样本生成的目标: y_pred
y_pred = knn.fit(X_train, y_train).predict(X_test)

def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

print('accuracy=%s' % accuracy(y_test, y_pred))

accuracy=0.94


# Linear regression and gradient_descent  

In [ ]:
Linear regression: y = ([1,...,1]T|X)w
损失函数L(w) =||Xw−y||22 , 最小化损失函数
两种方法 close form和 gradient descent 
close form: ŵ = (X.T * X).inv * X.T * y   
但在  X.T * X 是不可逆矩阵时 显式解不适用
寻找最小梯度,即梯度最大方向的反方向 #公式打不出来
alpha 为搜索步长，通过迭代更新梯度

In [2]:
import numpy as np
from sklearn.metrics import r2_score

class LinearRegression_:
    def __init__(self):
        self.w = None

    @staticmethod
    def ones_left(X):
        X = np.array(X)
        ones = np.ones(X.shape[0])
        return np.column_stack([ones, X])
    
    @staticmethod
    def closed_form(X ,y):
        product = np.dot(X.T, X)
        theInverse = np.linalg.inv(product)     #np.linalg.inv 即A的逆        显式解 : ŵ =(XTX)−1 * XTy
        return np.dot(np.dot(theInverse, X.T), y)
    
    @staticmethod
    def gradient_descent(X, y, n_iters=10000, alpha=0.004, weight=None):
        w = weight
        if w is None:
            w = np.random.rand(X.shape[1])
            w = np.ones(X.shape[1])
        pass
    
    
        for i in range(1, n_iters+1):        
            y_pred = X.dot(w)
            loss = y_pred - y
            
            grad = loss.dot(X)/X.shape[0]
            w = w - alpha *  grad         #使用迭代更新梯度  
            
            if i % (n_iters//10) == 0:
                print('iter:%d \ttraining MSE=%.3f \tMAE=%.3f \tr^2=%.3f'%(
                    i,
                    np.linalg.norm(loss),
                    np.linalg.norm(loss, ord=1),
                    r2_score(y, y_pred)
                    )
                )
        return w
    
    
    
    def fit(self, X_train, y_train, method='closed form', **kwargs):  # fit, 利用 X_train, y_train 计算  ŵ w^ , 
                                                                       #并将 ŵ ,w^ 更新到类的属性 self.w 中 

        X = self.ones_left(X_train)
        y = np.array(y_train)
        
        if method=='closed form':
            self.w = self.closed_form(X ,y)
        elif method == 'gradient descent':
            self.w = self.gradient_descent(X, y, **kwargs)
        return self

    
    def predict(self, X_test):
        X_test = np.array(X_test)
        augX_test = self.ones_left(X_test)
        return augX_test.dot(self.w)
    
# 测试
import numpy as np

mlr = LinearRegression_()

X = np.array([[1, 5], [3, 2], [6, 1]])
y = np.array([2, 3, 4])
y_pred = mlr.fit(X, y, method='gradient descent', 
                 n_iters=10000, 
                 alpha=0.05).predict(X)
print('fitted w is \t', mlr.w)
print('Am I correct? \t', np.isclose(y, y_pred, atol=1e-2).all())
print('training MSE=%.2f MAE=%.2f r^2=%.4f'% (
        np.linalg.norm(y_pred - y),
        np.linalg.norm(y_pred - y, ord=1),
        r2_score(y_pred, y)
    ))

iter:1000 	training MSE=0.172 	MAE=0.267 	r^2=0.985
iter:2000 	training MSE=0.085 	MAE=0.131 	r^2=0.996
iter:3000 	training MSE=0.042 	MAE=0.065 	r^2=0.999
iter:4000 	training MSE=0.021 	MAE=0.032 	r^2=1.000
iter:5000 	training MSE=0.010 	MAE=0.016 	r^2=1.000
iter:6000 	training MSE=0.005 	MAE=0.008 	r^2=1.000
iter:7000 	training MSE=0.002 	MAE=0.004 	r^2=1.000
iter:8000 	training MSE=0.001 	MAE=0.002 	r^2=1.000
iter:9000 	training MSE=0.001 	MAE=0.001 	r^2=1.000
iter:10000 	training MSE=0.000 	MAE=0.000 	r^2=1.000
fitted w is 	 [ 2.42720014  0.28592049 -0.14260796]
Am I correct? 	 True
training MSE=0.00 MAE=0.00 r^2=1.0000


# decision tree

In [ ]:
依照基尼不纯度的指标，计算每种分类的基尼系数，找到最佳分枝
如果使用树做回归, 即 y 为连续变量, 可以将impurity换成variance
这里略了binary tree, 剪枝部分我不完全明白因此也略掉了。

In [7]:
import pandas as pd
import numpy as np

credit = pd.read_excel('credit_risk.xlsx')
X = credit[['less2yrs', 'missedpay']].as_matrix()
y = credit['defaulted'].as_matrix()

#此决策树使用gini index 作为indicate，
def gini_impurity(y):
#   assume y are all integers
    p_i = np.bincount(y) / y.shape[0]
    return 1 - np.sum(p_i ** 2)


def gini_index(y_left, y_right):
    l_count, r_count = y_left.shape[0], y_right.shape[0]
    all_count = l_count + r_count
    
    l_imp = gini_impurity(y_left)
    r_imp = gini_impurity(y_right)
    
    return l_count/all_count * l_imp + r_count/all_count * r_imp


def get_best_split(X, y):
    min_gini_index = 1.0
    best_split = {}
    for i in range(X.shape[1]): 
        thresholds = np.unique(X[:, i])
        for th in thresholds:
            left_mask = X[:, i] <= th
            right_mask = X[:, i] > th

            gi = gini_index(y[left_mask], y[right_mask])
#             print('column=',i, 'thres=', th, 'gini_index=',gi)
            if gi < min_gini_index:
                min_gini_index = gi
                best_split = {'left mask':left_mask, 
                              'right mask':right_mask, 
                              'feature index':i, 
                              'threshold':th, 
                              'gini index':gi}
    return best_split


def leaf_label(y):
    return np.argmax(np.bincount(y))

class DecisionNode:
    def __init__(self, impurity=None):
        self.feature_index = None   # 从第几个feature分割数据集
        self.threshold = None       # 按照什么样的值分割数据集
        self.label = None           # 如果节点是叶子, node具有对应的标签
        self.true_branch = None     # 'Left' subtree
        self.false_branch = None    # 'Right' subtree
        self.impurity = impurity    # impurity最大是 1.0
    
    def __str__(self):
        return ' '.join(['imp=%s'%(self.impurity), 
                         'feature=%s'%(self.feature_index), 
                         'threshold=%s'%(self.threshold),
                         'label=%s'%(self.label),
                         '\n',
                         'left %s'%(self.truebranch),
                         'right %s'%(self.falsebranch)
                        ])
        
def build_tree(X, y):
    impurity = gini_impurity(y)
    node = DecisionNode(impurity=impurity)
    best_split = get_best_split(X, y)
    
    # 可以分叉
    if best_split['gini index'] < impurity:
        node.feature_index = best_split['feature index']
        node.threshold = best_split['threshold']
        left = best_split['left mask']
        right = best_split['right mask']
        node.truebranch = build_tree(X[left, :], y[left])
        node.falsebranch = build_tree(X[right, :], y[right])
    #不可再分, 叶子节点
    else:
        node.label = np.argmax(np.bincount(y))
    
    return node

def predict(tree, X):
    return [tree_predict(tree, row) for row in X]

def tree_predict(tree, x):
    # reached leaf
    if tree.label is not None:
        return tree.label
    
    prediction = None
    # node has branches
    # left? right?
    if x[tree.feature_index] <= tree.threshold:
        prediction = tree_predict(tree.truebranch, x)
    else:
        prediction = tree_predict(tree.falsebranch, x)
    return prediction
    

decisionTree = build_tree(X, y)
# print(decisionTree.truebranch)
y_pred = predict(decisionTree, X)
accuracy = (np.array(y_pred) ==  np.array(y) ).sum() / len(y)
accuracy


0.8